In [40]:
import pandas as pd

In [41]:
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]

In [42]:
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [43]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,...,hauteur_des_hanches,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille
1762,169.0116,35.0,69.000415,1.0,25.6032,62.0014,39.116000,99.8982,NaN,121.920000,...,83.3120,47.142400,46.6090,39.116000,138.04900,44.6024,81.8896,53.8988,86.7918,99.7966
1294,178.8922,58.0,73.935496,1.0,26.3906,67.2084,41.681400,98.2980,NaN,127.406400,...,94.2086,52.197000,43.6880,41.681400,148.79320,47.1932,86.7918,54.6100,87.4014,108.8898
782,181.7878,57.0,88.677236,1.0,26.5938,65.9892,40.919400,109.2962,NaN,132.410200,...,98.8060,51.104800,46.4058,40.919400,151.80310,45.9994,86.8934,59.5122,97.0026,110.0074
4338,180.5940,36.0,74.615884,1.0,26.3906,64.3890,41.249600,98.9076,NaN,129.590800,...,89.1032,48.996600,47.4980,41.249600,146.10080,46.9900,85.1916,55.4990,81.4070,102.3112
953,184.6072,58.0,91.698159,1.0,27.8892,67.9958,41.485936,106.7054,NaN,128.796549,...,88.5952,49.597886,51.3080,41.485936,145.86253,48.2092,89.9922,58.1914,98.6028,108.8898


In [44]:
caesar_male["largeur_des_hanches_ratio"] = caesar_male["largeur_des_hanches"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [46]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse']
    largeur_hanches = row['largeur_des_hanches_ratio']
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if largeur_hanches < 0.257434:
        cuisses = "fines"
    elif  0.257434 <=largeur_hanches<0.283177 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [47]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_male[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]
# Colonnes numériques et catégorielles
cols_num = ["taille", "weight", "age"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'weight', 'age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [48]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_tpot"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 22 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_des_hanches', 'largeur_des_hanches_ratio', 'largeur_du_dos', 'largeur_d_epaule', 'largeur_d_epaule_ratio', 'longueur_avant', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'sex', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_poitrine', 'tour_de_taille', 'tour_de_taille_ratio', 'tour_du_cou']


In [49]:
import os

dossier_modeles = "./modeles_tpot"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_complets"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_hanches_ratio.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_du_dos.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_d_epaule_ratio.pkl
✔ Pipelin

In [50]:
import os
import joblib
import pandas as pd

# 📁 1. Dossier où se trouvent les 22 pipelines
dossier_pipelines = "pipelines_complets"  # adapte selon ton cas

# 🧠 2. Charger tous les pipelines .pkl
pipelines = {}
for filename in os.listdir(dossier_pipelines):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        nom_variable = filename.replace("pipeline_tpot_", "").replace(".pkl", "")
        pipelines[nom_variable] = joblib.load(path)

# 📦 3. Exemple de nouvelles données
new_data = pd.DataFrame([{
    "taille": 187,
    "weight": 80,
    "age": 33,
    "categorie_ventre": "moyen",
    "categorie_cuisses": "medium",
    "categorie_torse": "large"
}])

# 🧮 4. Appliquer chaque pipeline à la même ligne de données
predictions = {}
for nom_variable, pipeline in pipelines.items():
    y_pred = pipeline.predict(new_data)
    predictions[nom_variable] = y_pred[0]  # extraire la valeur unique

# 🖨️ 5. Résultat
print("Prédictions :")
for var, val in predictions.items():
    print(f"{var} = {val:.2f}")


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\

Prédictions :
hauteur_des_epaules = 0.71
hauteur_des_genoux = 0.69
hauteur_des_hanches = 0.80
hauteur_de_la_taille = 0.95
hauteur_de_poitrine = 0.83
hauteur_d_entrejambe = 0.87
largeur_des_epaules_a_l_horizontales = 0.26
largeur_des_hanches = 0.26
largeur_des_hanches_ratio = -0.26
largeur_du_dos = 0.61
largeur_d_epaule = 0.61
largeur_d_epaule_ratio = -0.06
longueur_avant = 0.66
longueur_de_la_colonne_vertebrale_jusqu_au_poignet = 0.79
longueur_du_bras = 67.19
sex = 0.00
tour_de_cheville = 0.08
tour_de_cuisse = -0.39
tour_de_poitrine = -0.26
tour_de_taille = -0.29
tour_de_taille_ratio = -1.07
tour_du_cou = -0.29


In [33]:
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [51]:
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,...,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,largeur_des_hanches_ratio,tour_de_taille_ratio,largeur_d_epaule_ratio,categorie_ventre,categorie_torse,categorie_cuisses
1762,169.0116,35.0,69.000415,1.0,25.6032,62.0014,39.116000,99.8982,NaN,121.920000,...,81.8896,53.8988,86.7918,99.7966,0.263901,0.513526,0.231440,moyen,moyen,moyennes
1294,178.8922,58.0,73.935496,1.0,26.3906,67.2084,41.681400,98.2980,NaN,127.406400,...,86.7918,54.6100,87.4014,108.8898,0.263808,0.488570,0.232997,moyen,moyen,moyennes
782,181.7878,57.0,88.677236,1.0,26.5938,65.9892,40.919400,109.2962,NaN,132.410200,...,86.8934,59.5122,97.0026,110.0074,0.253039,0.533603,0.225094,moyen,moyen,fines
4338,180.5940,36.0,74.615884,1.0,26.3906,64.3890,41.249600,98.9076,NaN,129.590800,...,85.1916,55.4990,81.4070,102.3112,0.260197,0.450774,0.228411,plat,moyen,moyennes
953,184.6072,58.0,91.698159,1.0,27.8892,67.9958,41.485936,106.7054,NaN,128.796549,...,89.9922,58.1914,98.6028,108.8898,0.261145,0.534122,0.224725,moyen,fin,moyennes
